In [1]:
import cwltool

In [2]:
workflow = '/Users/dcl9/Code/python/mmap-cwl/go-blast/go-blasttool.cwl'

In [3]:
%%sh 
cwltool --print-pre /Users/dcl9/Code/python/mmap-cwl/go-blast/go-blasttool.cwl

{
    "cwlVersion": "https://w3id.org/cwl/cwl#draft-3.dev1", 
    "inputs": [
        {
            "type": "string", 
            "id": "file:///Users/dcl9/Code/python/mmap-cwl/go-blast/go-blasttool.cwl#db_name", 
            "description": "Prefix of the BLAST db files"
        }, 
        {
            "type": "File", 
            "id": "file:///Users/dcl9/Code/python/mmap-cwl/go-blast/go-blasttool.cwl#db_dir", 
            "description": "Directory containing the BLAST database"
        }, 
        {
            "default": "", 
            "inputBinding": {
                "position": 1, 
                "prefix": "-db", 
                "valueFrom": {
                    "engine": "file:///Users/dcl9/Code/python/mmap-cwl/py-expr-engine/py-expr-engine.cwl", 
                    "script": "os.path.join(self.job['db_dir']['path'], self.job['db_name'])\n"
                }
            }, 
            "type": "string", 
            "id": "file:///Users/dcl9/Code/python/mmap-cwl/go-blas

/Users/dcl9/Code/python/cwl-freeze/env/bin/cwltool 1.0.20151112194920


In [4]:
import yaml

In [5]:
y = None
with open(workflow) as f:
    y = yaml.load(f)

In [6]:
# find if y has a hint that is a docker requirement


In [7]:
def traverse(dic, path=None):
    if not path:
        path=[]
    if isinstance(dic,dict):
        for x in dic.keys():
            local_path = path[:]
            local_path.append(x)
            for b in traverse(dic[x], local_path):
                 yield b
    else: 
        yield path,dic

In [8]:
def dockerHints(dic):
    if 'hints' in dic:
        return dict['hints']
    return None

def dockerRequirements(dict):
    if 'requirements' in dic:
        return dict['requirements']
    return None

def imageIds(dic):
    if 'hints' in dic:
        for hint in dic['hints']:
            if hint['class'] == 'DockerRequirement':
                return hint['dockerImageId']
    return None




In [9]:
imageId(y)

NameError: name 'imageId' is not defined

In [10]:
import dpath.util

In [11]:
dpath.util.search(y,'hints/*/dockerImageId')

{'hints': [{'dockerImageId': 'dukegcb/go-blast'}]}

In [12]:
# Maybe I should use the schema? 

In [13]:
from schema_salad import schema

In [33]:
import urlparse
from cwltool import process, update
(document_loader, avsc_names, schema_metadata) = process.get_schema()

fileuri = 'file://' + workflow
workflowobj = document_loader.fetch(fileuri)
strict = True


In [36]:
# This updates from draft2 to draft3
workflowobj = update.update(workflowobj, document_loader, fileuri)
document_loader.idx.clear()

In [38]:
processobj, metadata = schema.load_and_validate(document_loader, avsc_names, workflowobj, strict)

In [44]:
type(processobj)

dict

## Questions

1. Could this be a CWL compiler?
2. WIll it take a root document and return the whole structure?
3. Can I find the dockerRequirement anywhere in the doc?